In [2]:
!ls -la ~/.keys

total 12
drwxrwxr-x  2 sweantu sweantu 4096 Aug 15 02:25 .
drwxr-x--- 15 sweantu sweantu 4096 Aug 15 08:23 ..
-rw-------  1 sweantu sweantu 2347 Aug 15 02:24 airbnb-468005-b68cd81995fd.json


In [6]:
!gcloud auth activate-service-account --key-file "/home/sweantu/.keys/airbnb-468005-b68cd81995fd.json"

Activated service account credentials for: [dev-898@airbnb-468005.iam.gserviceaccount.com]


In [7]:
!gsutil -m cp -r data/pq/ gs://airbnb-468005-bucket/spark_practice/data/pq

Copying file://data/pq/green/2021/08/.part-00000-e5f67910-386b-4781-89f9-d5ed14b9307b-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2021/08/part-00000-e5f67910-386b-4781-89f9-d5ed14b9307b-c000.snappy.parquet [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2021/08/._SUCCESS.crc [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2021/08/_SUCCESS [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2021/02/part-00003-20044390-1a4f-4a20-872e-86287fb7192b-c000.snappy.parquet [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2021/02/.part-00002-20044390-1a4f-4a20-872e-86287fb7192b-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2021/02/.part-00000-20044390-1a4f-4a20-872e-86287fb7192b-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://data/pq/green/2021/02/.part-00003-20044390-1a4f

In [9]:
!wget https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar

--2025-08-15 08:36:14--  https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.193.207, 173.194.194.207, 173.194.195.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.193.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40713341 (39M) [application/java-archive]
Saving to: ‘gcs-connector-hadoop3-latest.jar’

gcs-connector-hadoo 100%[===================>]  38.83M   133MB/s    in 0.3s    

2025-08-15 08:36:15 (133 MB/s) - ‘gcs-connector-hadoop3-latest.jar’ saved [40713341/40713341]



In [10]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [11]:
credentials_location = "/home/sweantu/.keys/airbnb-468005-b68cd81995fd.json"

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "gcs-connector-hadoop3-latest.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [12]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

25/08/15 08:37:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [13]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [14]:
df_green = spark.read.parquet('gs://airbnb-468005-bucket/spark_practice/data/pq/green/*/*')

In [15]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2021-06-18 13:31:15|  2021-06-18 13:31:24|                 N|         1|           7|           7|              1|          0.0|        2.5|  0.0|    0.

In [16]:
df_green.count()

570466

In [17]:
df_yellow = spark.read.parquet('gs://airbnb-468005-bucket/spark_practice/data/pq/yellow/*/*')

In [18]:
df_yellow.count()

15000700

In [19]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [20]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [21]:
common_columns = []
yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)

In [22]:
from pyspark.sql import functions as F

In [23]:
df_green_sel = df_green \
    .select(common_columns) \
    .withColumn('service_type', F.lit('green'))

In [24]:
df_yellow_sel = df_yellow \
    .select(common_columns) \
    .withColumn('service_type', F.lit('yellow'))

In [25]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [26]:
df_trips_data.groupBy('service_type').count().show()

[Stage 9:==================================================>        (6 + 1) / 7]

+------------+--------+
|service_type|   count|
+------------+--------+
|       green|  570466|
|      yellow|15000700|
+------------+--------+



In [27]:
df_trips_data.createOrReplaceTempView('trips_data')

In [28]:
spark.sql("""
SELECT * FROM trips_data LIMIT 10;
""").show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|service_type|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|       2|2021-06-18 13:31:15|2021-06-18 13:31:24|                 N|         1|           7|           7|              1|          0.0|        2.5|  0.0|    0.5|       0.0|         0.0|       

In [29]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY service_type;
""").show()

[Stage 13:=========================================>                (5 + 2) / 7]

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green|  570466|
|      yellow|15000700|
+------------+--------+



In [31]:
df_result = spark.sql("""
SELECT
    -- Revenue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month,
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,

    -- Additional calculations
    AVG(passenger_count) AS avg_monthly_passenger_count,
    AVG(trip_distance) AS avg_monthly_trip_distance
FROM trips_data
GROUP BY 1,2,3
""")

In [32]:
df_result.show()

[Stage 16:=================================================>        (6 + 1) / 7]

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|avg_monthly_passenger_count|avg_monthly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|         259|2021-04-01 00:00:00|       green|   7895.719999999992|                585.2|                    6.0|        14

In [34]:
df_result.coalesce(1).write.parquet('gs://airbnb-468005-bucket/spark_practice/data/report/revenue/', mode='overwrite')

In [35]:
df_report = spark.read.parquet('gs://airbnb-468005-bucket/spark_practice/data/report/revenue/')

In [36]:
df_report

DataFrame[revenue_zone: int, revenue_month: timestamp, service_type: string, revenue_monthly_fare: double, revenue_monthly_extra: double, revenue_monthly_mta_tax: double, revenue_monthly_tip_amount: double, revenue_monthly_tolls_amount: double, revenue_monthly_improvement_surcharge: double, revenue_monthly_total_amount: double, avg_monthly_passenger_count: double, avg_monthly_trip_distance: double]

In [37]:
df_report.count()

3713

In [38]:
df_report.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|avg_monthly_passenger_count|avg_monthly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|         259|2021-04-01 00:00:00|       green|   7895.719999999992|                585.2|                    6.0|        14